In [21]:
#! /usr/bin/env python
# -*- encoding: UTF-8 -*-
import cv2
import numpy as np 
from naoqi import ALProxy
import qi
import argparse
import sys
import Image
from matplotlib import pyplot as plt

nao_ip = "192.168.0.38"


navigation_service = ALProxy("ALNavigation", nao_ip, 9559)
motion_service = ALProxy("ALMotion", nao_ip, 9559)
tts_service = ALProxy("ALTextToSpeech", nao_ip, 9559)

motion_service.wakeUp() 

tts_service.setLanguage("English")
radius = 5
tts_service.say("Start exploration")
error_code = navigation_service.explore(radius)

if error_code != 0:
    print "Exploration failed."

path = navigation_service.saveExploration()

print "Exploration saved at path: \"" + path + "\""

navigation_service.startLocalization()
navigation_service.navigateToInMap([0., 0., 0.])
o_position = navigation_service.getRobotPositionInMap()
navigation_service.stopLocalization()

result_map = navigation_service.getMetricalMap()
map_width = result_map[1]
map_height = result_map[2]

o_x = round(o_position[0][1]/result_map[0] + result_map[3][1]/result_map[0])
o_y = round(o_position[0][0]/result_map[0] - result_map[3][0]/result_map[0])

img = np.array(result_map[4]).reshape(map_width, map_height)
img = (100 - img) * 2.55 # from 0..100 to 255..0
img = np.array(img, np.uint8)
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

tts_service.say("I got the map")

Exploration saved at path: "/home/nao/.local/share/Explorer/2019-03-28T140519.953Z.explo"


In [27]:
def get_location(event, x,y, flags, param):
    global ix,iy, drawing, mode, c_position

    if event == cv2.EVENT_FLAG_LBUTTON: 
        drawing = True
        cv2.circle(img_display, (x,y), 1, (255,0,0), 1)
#         cv2.imshow('image', img_display)
    elif event == cv2.EVENT_LBUTTONUP:
        x2m = y * result_map[0] + result_map[3][0]
        y2m = x * result_map[0] - result_map[3][1]
        print (x2m,y2m,x,y)
        tts_service.say("Start navigate to position" + str(round(x2m,2))+ "," +str(round(y2m,2)))
        
        navigation_service.startLocalization()
        navigation_service.navigateToInMap([x2m, y2m, 0.])        
        c_position = navigation_service.getRobotPositionInMap()
        navigation_service.stopLocalization()
        
        tts_service.say("navigation complete")
        print(c_position)
        

       
drawing = False 
mode = True 
ix,iy = -1,-1

img_display = img.copy()

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.setMouseCallback('image',get_location)

while True:
    drawing = True
    cv2.circle(img_display, (np.uint8(o_x),np.uint8(o_y)), 1, (0,0,255), 1)
    navigation_service.startLocalization()
    c_position = navigation_service.getRobotPositionInMap()
    navigation_service.stopLocalization()
    c_x = round(c_position[0][1]/result_map[0] + result_map[3][1]/result_map[0])
    c_y = round(c_position[0][0]/result_map[0] - result_map[3][0]/result_map[0])
    cv2.circle(img_display, (np.uint8(c_x),np.uint8(c_y)), 1, (0,255,0), 1)
    cv2.imshow('image', img_display) 
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cv2.destroyAllWindows()

(1.9874775856733322, 0.2208554446697235, 152, 124)
[[1.931947946548462, 0.3232106566429138, -0.15058374404907227], [0.15668506920337677, 0.07017847150564194, 0.08137251436710358]]
(-0.41252245008945465, -2.1791445910930634, 104, 76)
[[-0.5320714712142944, -1.7105404138565063, -1.9670641422271729], [3.479320526123047, 2.1006200313568115, 1.0286678075790405]]


RuntimeError: 	ALNavigation::startLocalization
	Session closed

In [18]:
result_map[3]


[-3.1526215076446533, 3.563805341720581]

In [11]:
90/result_map[3][0]

-14.877540538418682